### **Word2Vec**

원 핫 인코딩을 사용하면서도 단어 간 유사도를 반영할 수 있도록 단어의 의미를 벡터화하는 방법이다.

*비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다*는 분포 가설을 따르는 분산 표현 방법을 사용한다.

예) '강아지'는 주로 '귀엽다','예쁘다' 등의 단어와 함께 등장하는데, 이러한 내용을 가지는 텍스트를 벡터화하면 이 단어들은 의미적으로 가까운 단어가 된다.

<br>
Word2Vec에는 다음 두 가지 방식이 있다.  

- CBOW(Continous Bag of Words)
- Skip-Gram


#### **CBOW(Continous Bag of Words)**

주변에 있는 단어들을 통해 중간에 있는 단어를 예측하는 방법이다.  
윈도우를 두고, 윈도우 내의 주변 단어의 벡터로 중심 단어의 벡터를 예측한다.    
Skip-Gram에 비해 몇 배 빠른 훈련이 가능하며, 빈번한 단어를 예측하는 데 더 나은 정확도를 가진다.

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import os
import re

In [7]:
BASE_DIR = "/data/TestDir/sample_articles"
ORIGIN_PATH = os.path.join(BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(BASE_DIR,"Preprocessed-Train-Data")
PRETTY_PATH = os.path.join(BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [8]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

경향신문 언론사 첫번째 기사를 예시로 전처리된 텍스트를 토큰화 한다.

In [9]:
media_list = os.listdir(ORIGIN_PATH)

media = media_list[2]
media_path = os.path.join(PREPROCESSED_PATH, media)
article_list= os.listdir(media_path)

article = article_list[0]
reader = RawTextReader(os.path.join(media_path, article))
    
content = list(filter(None, reader))
content

['진행',
 '강진원 앵커 박상연 앵커 출연',
 '김광삼 변호사 차재원 부산가톨릭대 특임교수 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다',
 '대법원의 선고 결과 이재명 경기지사와 관련한 대법원의 선고 결과 관련해서 전문가와 함께 자세한 내용 짚어보겠습니다',
 '변호사님 일단 먼저 주문을 있는 그대로 제가 한번 읽어드릴게요',
 '다수의견에 따라서 다음과 같이 판결한다',
 '원심 판결 중 유무죄 부분을 포함한 유죄 부분을 파기하고 이 부분 사건을 수원고등법원에 환송한다',
 '결국은 허위사실공표 관련된 내용을 무죄 취지로 파기환송한다 이렇게 봐야겠죠',
 '정확히 말씀하셨고요',
 '무죄 취지로 파기환송을 했기 때문에 이건 다시 서울고등법원에서 재판을 다시 하게 됩니다',
 '그런데 대법원에서 이미 무죄 취지였기 때문에 서울고등은 그 취지에 귀속이 될 수밖에 없어요',
 '그러면 결과적으로 이재명 지사는 수원고등에서 무죄판결을 받게 될 것이다 이렇게 볼 수 있고요',
 '거기에 대해서 만약에 검찰이 재상고를 하게 되면 대법원에서 다시 재판을 받게 되겠지만 그건 그렇게 결과에 영향을 미치는 그런 선고는 나오지 않을 것이다 이렇게 볼 수 있는 것이죠',
 '실익이 없는 거죠 검찰 입장에서는',
 '맞습니다',
 '사실 일각에서는 이 사건이 전원합의체로 넘어간 것 자체가 이 지사에게 긍정적일 수 있다 이런 시각이 있지 않았습니까',
 '맞습니다',
 '전원합의체로 넘어갔고 그리고 또 이례적으로 생중계를 대법원에서 결정하지 않았습니까',
 '그렇기 때문에 많은 전문가들은 아마 2심의 결과하고 반대되는 결과를 냈기 때문에 상당히 그런 차원에서 대법원이 이런 부분을 널리 홍보하고 싶었던 것이 아닐까 하는 생각이 들었다는 것이고 그리고 또 오늘 전원합의체의 다수 의견 자체가 허위사실공표죄의 새로운 판례를 확립했다',
 '그런 측면에서라도 이런 부분들을 적극적으로 알리고 싶어 하는 그런 부분들이 반영됐을 것

In [10]:
tokenList = [sent.split() for sent in content]
tokenList

[['진행'],
 ['강진원', '앵커', '박상연', '앵커', '출연'],
 ['김광삼',
  '변호사',
  '차재원',
  '부산가톨릭대',
  '특임교수',
  '아래',
  '텍스트는',
  '실제',
  '방송',
  '내용과',
  '차이가',
  '있을',
  '수',
  '있으니',
  '보다',
  '정확한',
  '내용은',
  '방송으로',
  '확인하시기',
  '바랍니다'],
 ['대법원의',
  '선고',
  '결과',
  '이재명',
  '경기지사와',
  '관련한',
  '대법원의',
  '선고',
  '결과',
  '관련해서',
  '전문가와',
  '함께',
  '자세한',
  '내용',
  '짚어보겠습니다'],
 ['변호사님', '일단', '먼저', '주문을', '있는', '그대로', '제가', '한번', '읽어드릴게요'],
 ['다수의견에', '따라서', '다음과', '같이', '판결한다'],
 ['원심',
  '판결',
  '중',
  '유무죄',
  '부분을',
  '포함한',
  '유죄',
  '부분을',
  '파기하고',
  '이',
  '부분',
  '사건을',
  '수원고등법원에',
  '환송한다'],
 ['결국은', '허위사실공표', '관련된', '내용을', '무죄', '취지로', '파기환송한다', '이렇게', '봐야겠죠'],
 ['정확히', '말씀하셨고요'],
 ['무죄',
  '취지로',
  '파기환송을',
  '했기',
  '때문에',
  '이건',
  '다시',
  '서울고등법원에서',
  '재판을',
  '다시',
  '하게',
  '됩니다'],
 ['그런데',
  '대법원에서',
  '이미',
  '무죄',
  '취지였기',
  '때문에',
  '서울고등은',
  '그',
  '취지에',
  '귀속이',
  '될',
  '수밖에',
  '없어요'],
 ['그러면',
  '결과적으로',
  '이재명',
  '지사는',
  '수원고등에서',
  '무죄판결을',
  '받게',
  '될',
  '것이다',

#### **Word2Vec 학습**

gensim에서는 다음과 같이 Word2Vec을 지원한다.  

> `sentences` : 단어 토큰화된 문장
> `size` : Projection Layer의 크기, 임베딩 벡터의 차원  
> `window` : 윈도우 크기  
> `min_count` : 단어의 최소 빈도 수, 이 이하의 빈도를 가지는 단어는 학습하지 않음  
> `workers` : 학습을 위한 프로세스 수  
> `sg` : 0일 경우 CBOW, 1일 경우 Skip-Gram  



In [13]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=tokenList, size=100, window=5, min_count=1, workers=4, sg=0)

'선거'를 입력했을 때 의미가 유사한 단어들을 출력합니다.

In [14]:
model_result = model.wv.most_similar("선거")
model_result

[('뉴스에서', 0.338172972202301),
 ('귀속이', 0.3328704535961151),
 ('표현했다고', 0.33212876319885254),
 ('저는', 0.3084290623664856),
 ('결국은', 0.29316264390945435),
 ('그렇죠', 0.28543901443481445),
 ('말씀드린', 0.2808339595794678),
 ('대선주자와', 0.27329280972480774),
 ('지지층들과의', 0.26043033599853516),
 ('전원합의체에', 0.25079941749572754)]

#### **Skip-Gram**

중심 단어를 통해 주변에 있는 단어들을 예측하는 방법이다.  
소량의 학습 데이터에서도 잘 동작하며, 자주 사용하지 않는 희귀한 단어를 예측할 수 있다. 하지만 계산 비용이 크다는 문제점이 있다.   
마찬가지로 중심 단어에 윈도우를 두고, 윈도우 내의 주변 단어의 임베딩 벡터를 예측한다.

In [15]:
model = Word2Vec(sentences=tokenList, size=100, window=5, min_count=1, workers=4, sg=1)

In [16]:
model_result = model.wv.most_similar("선거")
model_result

[('뉴스에서', 0.35526594519615173),
 ('표현했다고', 0.3478245735168457),
 ('귀속이', 0.34415918588638306),
 ('저는', 0.33563393354415894),
 ('그렇죠', 0.29303207993507385),
 ('대선주자와', 0.292006254196167),
 ('결국은', 0.29079103469848633),
 ('말씀드린', 0.28423285484313965),
 ('전원합의체에', 0.2752193212509155),
 ('지지층들과의', 0.2710190713405609)]

기사 본문 내용이 짧기 때문에, 모델을 학습하기에 corpus의 크기가 작다.  
유사도 또한 상당히 낮다.   
아래는 수집한 기사 100,000건들을 통해 corpus를 구성하고, Word2Vec 모델을 구축하는 내용이다.

In [17]:
media_list = os.listdir(ORIGIN_PATH)

result = []
forCount = []
for media in media_list:
    media_path = os.path.join(PREPROCESSED_PATH, media)
    article_list= os.listdir(media_path)

    for article in article_list:
        reader = RawTextReader(os.path.join(media_path, article)) 
        content = list(filter(None, reader))
        forCount += [token for sent in content for token in sent.split()]
        result += [sent.split() for sent in content]

In [18]:
print("전체 token의 개수 : {len}".format(len=len(forCount)))
print("중복되지 않은 token의 개수 : {len}".format(len=len(list(set(forCount)))))

전체 token의 개수 : 22341223
중복되지 않은 token의 개수 : 1107954


In [ ]:
model = Word2Vec(sentences=result, size=200, window=5, min_count=3, workers=4, sg=1)

In [ ]:
model_result = model.wv.most_similar("선거")
model_result

In [ ]:
model.wv.__getitem__('선거')

In [ ]:
model.save("word2vec.model")